In [19]:
import re
import time
import pytz
import random
import datetime
import requests
from bs4 import BeautifulSoup

# 填写你的问卷号
questionnaire_number = 

# 这个是问卷星里面的"来源" directphone显示为手机提交, 微信显示微信提交, http://www.wjx.cn/为网页访问
# directphone  微信  http://www.wjx.cn/
source = 'directphone'

# 问卷内容 意思为第一题选B 第二题选A 第三题选B 第四题选B 第五题选B
submitdata = "1$2}2$1}3$2}4$2}5$2"

def send():
    # 问卷地址
    url = "https://www.wjx.cn/jq/{}.aspx".format(questionnaire_number)

    payload = ""
    headers = {
        'cache-control': "no-cache",
    }
    
    # 开始请求
    response = requests.request("GET", url, data=payload, headers=headers)
    setCookie = response.headers['Set-Cookie']
    getCookie = re.findall(r'acw_tc=.*?;', setCookie)[0] + re.findall(r'\.ASP.*?;', setCookie)[0] + re.findall(r'jac.*?;', setCookie)[0]
    CookieText = getCookie
    html_soup = BeautifulSoup(response.text, "html.parser")
    pubDate = html_soup.find('script', {'type': 'text/javascript'})
    
    # 查找curID, 其实就是问卷号
    curid_reg = re.compile(r'activityId = ?\'?(\d+?)\'?;')
    curID = curid_reg.search(response.text).group(1)
    
    # ramdon number, 没研究是什么
    rnnum_reg = re.compile(r'rndnum="(\d+.?\d+?)";')
    rnnum = rnnum_reg.search(response.text).group(1)
    
    # 请求UUID
    jqnonce_reg = re.compile(r'[0-9a-z]{8}-[0-9a-z]{4}-[0-9a-z]{4}-[0-9a-z]{4}-[0-9a-z]{12}')
    jqnonce = jqnonce_reg.search(response.text).group(0)
    
    # 回答完问卷的时间 UNIX时间戳 毫秒级别
    t = str(int(time.time()*1000)+ 28800 - 60)
    
    # 开始答问卷的时间
    starttime = (datetime.datetime.now()+datetime.timedelta(hours=8)-datetime.timedelta(seconds=random.randint(20,40))).strftime('%Y-%m-%d %H:%M:%S')
    
    # 回答问卷所用的时间, 随机生成45至200秒
    ktimes = random.randint(45,200)
    
    # 关键加密算法破解, jqsign与ktimes有关
    a = ktimes%10
    if a == 0:
        a = 1
    
    # 生成jqsign, 问卷星骚操作
    jqsign_list = list()
    for i in range(len(jqnonce)):
        e = ord(jqnonce[i]) ^ a
        jqsign_list.append(chr(e))
    jqsign_str = ''.join(jqsign_list)

    querystring = {"submittype":"1" ,"hlv":"1"}
    
    # curID, 其实就是问卷号
    querystring['curID'] = curID
    
    # 回答问卷所用的时间, 随机生成45至200秒
    querystring['ktimes'] = ktimes
    
    # 回答完问卷的时间 UNIX时间戳 毫秒级别
    querystring['t'] = t
    
    # 开始答问卷的时间
    querystring['starttime'] = starttime
    
    # ramdon number, 没研究是什么
    querystring['rn'] = rnnum
    
    # 请求UUID
    querystring['jqnonce'] = jqnonce
    
    # 加密字符
    querystring['jqsign'] = jqsign_str
    
    # 来源
    querystring['source'] = source

    # 开始post请求
    post_url = "https://www.wjx.cn/joinnew/processjq.ashx"
    payload = "------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"submitdata\"\r\n\r\n%s\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW--" % submitdata
    headers = {
        'accept': '*/*',
        'Host': 'www.wjx.cn',
        'Connection': 'keep-alive',
        'Content-Length': '156',
        'Origin': 'https://www.wjx.cn',
        'content-type': "multipart/form-data; boundary=----WebKitFormBoundary7MA4YWxkTrZu0gW",
        'cache-control': "no-cache",
        'referer': 'https://www.wjx.cn/m/{}.aspx'.format(questionnaire_number),
        'User-Agent':'Mozilla/5.0 (Linux; U; Android 4.3; en-us; SM-N900T Build/JSS15J) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 Mobile Safari/534.30',
        'x-requested-with': 'XMLHttpRequest',
        "Cookie": CookieText,
        }
    response = requests.request("POST", post_url, data=payload, headers=headers, params=querystring)
    if(re.findall(r'complete', response.text)):
        print("成功!")
        print(response.text)
    else:
        print("Error")
        print(response.text)
        
send()

成功!
10〒/wjx/join/complete.aspx?q=隐藏&JoinID=隐藏&jidx=隐藏
